In [40]:
import os
import time,datetime
import math
import numpy as np
from astropy.io import fits
from astropy.visualization import simple_norm
from photutils import CircularAnnulus
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
nullfmt   = NullFormatter()         # no labels

COLOR = 'k'#'#FFFAF1'
plt.rcParams['font.size'] = 9
plt.rcParams['text.color'] = COLOR
plt.rcParams['axes.labelcolor'] = COLOR
plt.rcParams['xtick.color'] = COLOR
plt.rcParams['ytick.color'] = COLOR

plt.rcParams['xtick.major.width'] = 1.5
plt.rcParams['ytick.major.width'] = 1.5
plt.rcParams['xtick.major.size']  = 5 #12
plt.rcParams['ytick.major.size']  = 5#12

plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.minor.width'] = 1
plt.rcParams['xtick.minor.size']  = 8
plt.rcParams['ytick.minor.size']  = 8

plt.rcParams['axes.linewidth'] = .5

plt.rcParams['text.color'] = COLOR
plt.rcParams['xtick.color'] = COLOR
plt.rcParams['ytick.color'] = COLOR
plt.rcParams['axes.labelcolor'] = COLOR
#plt.rcParams['axes.spines.top'] = False
#plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.labelcolor'] = COLOR
plt.rcParams['axes.edgecolor'] = COLOR
plt.rcParams['figure.facecolor'] = 'none'
plt.rcParams['legend.facecolor'] = 'none'
from matplotlib.gridspec import GridSpec
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from mpl_toolkits.axes_grid1 import make_axes_locatable


/var/folders/tf/hszq26_12053sq83srkrc3m40000gn/T/ipykernel_8535/4026823169.py:7: DeprecationWarning: `photutils.CircularAnnulus` is a deprecated alias for `photutils.aperture.CircularAnnulus` and will be removed in the future. Instead, please use `from photutils.aperture import CircularAnnulus` to silence this warning.
  from photutils import CircularAnnulus


In [95]:
def plot_hyperstack_diagram(base_path):
    plot_filepath = base_path + 'fig_hyperstack_diagrams.pdf'
    
    # Define panel positions/sizes
#     rect_plot = {'obj': {'as': {'image':[0.120,0.575,0.150,0.150],'wedge_diagram':[0.275,0.575,0.150,0.150]},
#                          'nhv':{'image':[0.120,0.255,0.150,0.150],'wedge_diagram':[0.275,0.255,0.150,0.150]}},
#                  'star':{'as': {'image':[0.120,0.420,0.150,0.150],'wedge_diagram':[0.275,0.420,0.150,0.150]},
#                          'nhv':{'image':[0.120,0.100,0.150,0.150],'wedge_diagram':[0.275,0.100,0.150,0.150]}}}
    rect_plot = {'obj': {'as': {'image':[0.00,0.6,0.190,0.190],'wedge_diagram':[0.2,0.6,0.190,0.190]},
                         'nhv':{'image':[0.00,0.2,0.190,0.190],'wedge_diagram':[0.2,0.2,0.190,0.190]}},
                 'star':{'as': {'image':[0.00,0.4,0.190,0.190],'wedge_diagram':[0.2,0.40,0.190,0.190]},
                         'nhv':{'image':[0.00,0.00,0.190,0.190],'wedge_diagram':[0.2,0.00,0.190,0.190]}}}
#     axPlot    = {'obj': {'as': {'image':0,'img_overlay':0,'wedge_diagram':0},
#                          'nhv':{'image':0,'img_overlay':0,'wedge_diagram':0}},
#                  'star':{'as': {'image':0,'img_overlay':0,'wedge_diagram':0},
#                          'nhv':{'image':0,'img_overlay':0,'wedge_diagram':0}}}
    axPlot    = {'obj': {'as': {'image':[0.00,0.6,0.190,0.190],'img_overlay':[0.00,0.6,0.190,0.190],'wedge_diagram':[0.00,0.6,0.190,0.190]},
                         'nhv':{'image':[0.00,0.6,0.190,0.190],'img_overlay':[0.00,0.6,0.190,0.190],'wedge_diagram':[0.00,0.6,0.190,0.190]}},
                 'star':{'as': {'image':[0.00,0.6,0.190,0.190],'img_overlay':[0.00,0.6,0.190,0.190],'wedge_diagram':[0.00,0.6,0.190,0.190]},
                         'nhv':{'image':[0.00,0.6,0.190,0.190],'img_overlay':[0.00,0.6,0.190,0.190],'wedge_diagram':[0.00,0.6,0.190,0.190]}}}             
    # Define panel text
    letter_label    = {'obj': {'as':'(a)','nhv':'(c)'},'star':{'as':'(b)','nhv':'(d)'}}
    image_label     = {'obj':'Targeted objects','star':'Reference stars'}
    alignment_label = {'as':r'-$\odot$','nhv':r'-$v$'}
    
    # Set figure size
    cm = 1/2.54  # centimeters in inches
    plt.figure(1,figsize=(8.7*cm,10*cm),tight_layout=False)#,gridspec_kw={'hspace': 0.35, 'wspace': 0.3})
    #plt.figure(1,figsize=(16,16))
    #fig, axs = plt.subplots(1, 1, tight_layout=False,figsize=(8.7*cm,8.7*cm),sharex=False,sharey=False
                      # ,gridspec_kw={'hspace': 0.35, 'wspace': 0.3})

    # Define formatting presets
    props1          = dict(boxstyle='round,pad=0.05',facecolor='none',alpha=0.8,edgecolor='none')
    props2          = dict(boxstyle='round,pad=0.10',facecolor='none',alpha=0.8,edgecolor='none')
    fs=6
    linewidth=0.5
    for target in ['obj','star']:
        for alignment in ['as','nhv']:
            # Create image panels
            axPlot[target][alignment]['image']       = plt.axes(rect_plot[target][alignment]['image'])
            axPlot[target][alignment]['img_overlay'] = plt.axes(rect_plot[target][alignment]['image'])
            image_filepath = base_path + 'darkcomets.hyperstack.median.{:s}.bgsub.rot.{:s}.png'.format(target,alignment)
            image = plt.imread(image_filepath)
            axPlot[target][alignment]['image'].imshow(image)
            axPlot[target][alignment]['img_overlay'].text(0.03,0.800,letter_label[target][alignment],fontsize=6,transform=axPlot[target][alignment]['image'].transAxes,bbox=props2,zorder=2)
            axPlot[target][alignment]['img_overlay'].set_xlim([-1,1])
            axPlot[target][alignment]['img_overlay'].set_ylim([-1,1])
            axPlot[target][alignment]['image'].xaxis.set_major_formatter(nullfmt)
            axPlot[target][alignment]['image'].yaxis.set_major_formatter(nullfmt)
            axPlot[target][alignment]['image'].tick_params(axis='both',which='both',bottom=False,top=False,left=False,right=False)
            axPlot[target][alignment]['img_overlay'].xaxis.set_major_formatter(nullfmt)
            axPlot[target][alignment]['img_overlay'].yaxis.set_major_formatter(nullfmt)
            axPlot[target][alignment]['img_overlay'].tick_params(axis='both',which='both',bottom=False,top=False,left=False,right=False)
            axPlot[target][alignment]['img_overlay'].arrow(0.3,0.0,0.4,0.0,head_width=0.1,head_length=0.1,lw=linewidth,ec='black',fc='black')
            axPlot[target][alignment]['img_overlay'].text(0.43,0.07,alignment_label[alignment],fontsize=4,bbox=props1,zorder=2)
            axPlot[target][alignment]['img_overlay'].text(0.25,0.05,image_label[target],fontsize=4,transform=axPlot[target][alignment]['image'].transAxes,bbox=props2,zorder=2)
            axPlot[target][alignment]['img_overlay'].axis('off')
            # Generate wedge diagram panels
            axPlot[target][alignment]['wedge_diagram'] = plt.axes(rect_plot[target][alignment]['wedge_diagram'])
            fits_image_filepath = base_path + 'darkcomets.hyperstack.median.{:s}.bgsub.rot.{:s}.0.fits'.format(target,alignment)
            
            # Read in FITS data
            with fits.open(fits_image_filepath) as hdu:
                data_yx_adu       = hdu[0].data
                numpix_y,numpix_x = data_yx_adu.shape

            # Set wedge diagram parameters
            pixscale               = 0.2637
            central_radius_arcsec  = 4
            radius_inner_arcsec    = 5
            radius_outer_arcsec    = 15
            angle_interval_deg     = 15
            start_angle            = 0
            image_half_size_arcsec = radius_outer_arcsec + 1.5
            image_half_size_pix    = image_half_size_arcsec / pixscale
            radius_inner_pix       = radius_inner_arcsec/pixscale
            radius_outer_pix       = radius_outer_arcsec/pixscale
            positions              = [(int((numpix_x-1)/2),int((numpix_y-1)/2))]

            # Compute individual wedge parameters
            num_wedges          = int(360 / angle_interval_deg)
            wedge_angle_start   = [0 for idx in range(num_wedges)]
            wedge_angle_end     = [0 for idx in range(num_wedges)] 
            wedge_az_angle      = [0 for idx in range(num_wedges)] 
            for idx in range(num_wedges):
                wedge_angle_start[idx] = start_angle + (angle_interval_deg * idx) - angle_interval_deg / 2
                wedge_angle_end[idx]   = start_angle + (angle_interval_deg * (idx+1)) - angle_interval_deg / 2
                wedge_az_angle[idx]    = (start_angle + (angle_interval_deg * idx)) % 360

            # Create wedge diagram
            annulus_aperture = CircularAnnulus(positions,r_in=radius_inner_pix,r_out=radius_outer_pix)
            for idx in range(len(wedge_angle_start)):
                x_start = int((numpix_x-1)/2) + radius_inner_pix * math.cos((wedge_angle_start[idx]+90)%360/180*math.pi)
                y_start = int((numpix_y-1)/2) + radius_inner_pix * math.sin((wedge_angle_start[idx]+90)%360/180*math.pi)
                x_end   = int((numpix_x-1)/2) + radius_outer_pix * math.cos((wedge_angle_start[idx]+90)%360/180*math.pi)
                y_end   = int((numpix_y-1)/2) + radius_outer_pix * math.sin((wedge_angle_start[idx]+90)%360/180*math.pi)
                xcoords = [x_start,x_end]
                ycoords = [y_start,y_end]
                plt.plot(xcoords,ycoords,color='#ff0000',lw=linewidth,ls='-')
            ann_patches = annulus_aperture.plot(color='red',lw=linewidth,label='Background annulus')

            # Display background image
            norm = simple_norm(data_yx_adu,'sqrt',percent=99)
            plt.imshow(data_yx_adu,cmap='gray',norm=norm,interpolation='nearest')
            plt.xlim(int((numpix_x-1)/2)-image_half_size_pix,int((numpix_x-1)/2)+image_half_size_pix)
            plt.ylim(int((numpix_y-1)/2)-image_half_size_pix,int((numpix_y-1)/2)+image_half_size_pix)

            # Turn off axis labels and ticks
            axPlot[target][alignment]['wedge_diagram'].xaxis.set_major_formatter(nullfmt)
            axPlot[target][alignment]['wedge_diagram'].yaxis.set_major_formatter(nullfmt)
            axPlot[target][alignment]['wedge_diagram'].tick_params(axis='both',which='both',bottom=False,top=False,left=False,right=False)
            
    plt.savefig(plot_filepath,format='pdf',transparent=True,dpi=600,bbox_inches='tight')
    #plt.savefig('Figure10_Observability_Vector.pdf',
     #       format='pdf',bbox_inches='tight', dpi=600)
    plt.clf()
    plt.cla()
    plt.close()

    print('{:s} - >> Plotting complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    return None


In [96]:
base_path = '/Users/darrylseligman/Dark_Comets/NewDarkComets/FinalFigures/hyperstack_diagram_plotting/'
plot_hyperstack_diagram(base_path)


2024-07-10 06:56:04 - >> Plotting complete.


In [30]:
pwd

'/Users/darrylseligman/Dark_Comets/NewDarkComets/FinalFigures/hyperstack_diagram_plotting'

In [31]:
ls

darkcomets.hyperstack.median.obj.bgsub.rot.as.0.fits
darkcomets.hyperstack.median.obj.bgsub.rot.as.png
darkcomets.hyperstack.median.obj.bgsub.rot.nhv.0.fits
darkcomets.hyperstack.median.obj.bgsub.rot.nhv.png
darkcomets.hyperstack.median.star.bgsub.rot.as.0.fits
darkcomets.hyperstack.median.star.bgsub.rot.as.png
darkcomets.hyperstack.median.star.bgsub.rot.nhv.0.fits
darkcomets.hyperstack.median.star.bgsub.rot.nhv.png
fig_hyperstack_diagrams.pdf
notebook_plot_hyperstack_diagram.ipynb
